```{julia}
using Distributions
using DataFrames
using Chain
using CairoMakie
using StatisticalRethinking
using CSV
```


## 4.1 Why normal distributions are normal

### 4.1.1 Normal by addition


```{julia}
pos = [sum(rand(Uniform(-1, 1), 16)) for _ in 1:1000]
pos = map(i -> sum(rand(Uniform(-1, 1), 16)), 1:1000)
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

density!(ax, pos)
f
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])
x_range = 1:16

for _ in 1:1000
    movements = cumsum(rand(Uniform(-1, 1), 16))
    lines!(ax, x_range, movements, color = (:black, 0.25))
end

f
```


### 4.1.2 Normal by multiplication


```{julia}
prod(1 .+ rand(Uniform(0, 0.1), 12))
```

```{julia}
growth = [prod(1 .+ rand(Uniform(0, 0.1), 12)) for _ in 1:10_000]
f = Figure()
ax = Axis(f[1, 1])
density!(ax, growth)
f
```

```{julia}
big = [prod(1 .+ rand(Uniform(0, 0.5), 12)) for _ in 1:10_000]
small = [prod(1 .+ rand(Uniform(0, 0.1), 12)) for _ in 1:10_000]
```


### 4.1.3 Normal by log-multiplication

```{julia}
log_big = [log(prod(1 .+ rand(Uniform(0, 0.5), 12))) for _ in 1:10_000]
```


### 4.1.4 Using Gaussian distributions

## 4.2 A language for describing models

### 4.2.1 Re-describing the globe tossing model


```{julia}
w = 6
n = 9
p_grid = range(start=0, stop=1, length=100)
posterior = pdf.(Binomial.(n, p_grid), w) .* pdf.(Uniform(0, 1), p_grid)
posterior = posterior / sum(posterior)
```


## 4.3 Gaussian model of height

### 4.3.1 The data


```{julia}
howell1 = CSV.read("Howell1.csv", DataFrame; delim=";")
```

```{julia}
describe(howell1)
```

```{julia}
precis(howell1)
```

```{julia}
howell1[:, :height]
```

```{julia}
d2 = subset(howell1, :age => ByRow(age -> age .>= 18))
```


### 4.3.2 The model


```{julia}
avg_height = 100:250
prior = Normal(178, 20)
likelihood = pdf.(prior, avg_height)

f = Figure()
ax = Axis(f[1, 1])
lines!(ax, avg_height, likelihood)
f
```

```{julia}
avg_spread = -10:60
prior = Uniform(0, 50)
likelihood = pdf.(prior, avg_spread)

f = Figure()
ax = Axis(f[1, 1])
lines!(ax, avg_spread, likelihood)
f
```

```{julia}
sample_mu = rand(Normal(178, 20), 10_000)
sample_sigma = rand(Uniform(0, 50), 10_000)
prior_h = rand.(Normal.(sample_mu, sample_sigma))
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])
density!(ax, prior_h)
f
```

```{julia}
sample_mu = rand(Normal(178, 100), 10_000)
prior_h = rand.(Normal.(sample_mu, sample_sigma))
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])
density!(ax, prior_h)
f
```


### 4.3.3 Grid approximation of the posterior distribution

```{julia}
mu_list = range(start=150, stop=160, length=100)
sigma_list = range(start=7, stop=9, length=100)
post = DataFrame(Iterators.product(mu_list, sigma_list), [:mu, :sigma])
post[:, :log_likelihood] = map(
    (mu, sigma) -> sum(logpdf.(Normal(mu, sigma), d2[:, :height])),
    post[:, :mu], post[:, :sigma]
)
post[:, :product] = post[:, :log_likelihood] + 
                    logpdf.(Normal(178, 20), post[:, :mu]) +
                    logpdf.(Uniform(0, 50), post[:, :sigma])
# NOTE: This is the unnormalized posterior!
post[:, :prob] = exp.(post[:, :product] .- maximum(post[:, :product]))
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

contour!(ax, post[:, :mu], post[:, :sigma], post[:, :prob])
f
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

heatmap!(ax, post[:, :mu], post[:, :sigma], post[:, :prob])
f
```


### 4.3.4 Sampling from the posterior


```{julia}
sample_rows = sample(1:nrow(post), Weights(post[:, :prob]), 10_000, replace=true)
sample_mu = post[sample_rows, :mu]
sample_sigma = post[sample_rows, :sigma]
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

scatter!(ax, sample_mu, sample_sigma, color = (:blue, 0.15))
f
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

density!(ax, sample_mu)
f
```

```{julia}
f = Figure()
ax = Axis(f[1, 1])

density!(ax, sample_sigma)
f
```

```{julia}
PI(sample_mu)
PI(sample_sigma)
```

```{julia}
d3 = sample(d2[:, :height], 20)
```

```{julia}
mu_list = range(start=150, stop=170, length=200)
sigma_list = range(start=4, stop=20, length=200)
post2 = DataFrame(Iterators.product(mu_list, sigma_list), [:mu, :sigma])

post2[:, :]
```